# Install and import lib

In [1]:
from tweety import Twitter
import asyncio
import os
import csv
import tweety
import time


# Login


In [2]:

auth_token = os.getenv("auth_token1")

# Cookies can be a str or a dict

app = Twitter("session")
app.load_auth_token(auth_token)

<coroutine object AuthMethods.load_auth_token at 0x00000228400BBDE0>

In [3]:
async def login():
    # Read username and password from environment variables
    username = ""
    password = ""

    if not username or not password:
        raise ValueError("USERNAMEE or PASSWORD is not configured in the .env file")

    # Initialize Twitter client
    app = Twitter("session")
    await app.sign_in(username, password)
    return app

In [4]:
app = await login()

In [5]:
from tweety.types import twDataTypes

async def crawl_followers_usernames(input_file_path, output_file_path, app, pages=1, wait_time=2):
    kol_followers_data = []

    # Read the input file and get the list of screen names
    with open(input_file_path, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        screen_names = [row['twitter_id'] for row in reader]
        screen_names = screen_names[:500] #thay số chỗ nà
    
    # Open the error file to log users with rate limit errors
    with open("crawl_user_followers_error.txt", mode="a", encoding="utf-8") as error_file:
        # Loop through each KOL username to fetch their followers
        for screen_name in screen_names:
            print(f"Fetching followers for user: {screen_name}")
            retry_count = 0
            while retry_count <= 3:  # Retry up to 3 times
                try:
                    # Fetch followers using the Tweety app
                    all_followers = await app.get_user_followers(
                        username=screen_name,
                        pages=pages,
                        wait_time=wait_time
                    )
                    # Collect only the follower usernames
                    followers_usernames = [follower.username for follower in all_followers]
                    kol_followers_data.append({
                        'KOL_username': screen_name,
                        'followers_usernames': followers_usernames
                    })
                    break  # Exit retry loop if successful
                except Exception as e:
                    retry_count += 1
                    print(f"Error fetching followers for user {screen_name}: {e}. Retrying ({retry_count}/3)...")
                    if retry_count > 3:
                        error_file.write(f"{screen_name}\n")
                    time.sleep(300)  # Wait before retrying

    # Save the results to the output file in a structured format
    with open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['KOL_username', 'followers_usernames'])  # Write the header

        for data in kol_followers_data:
            writer.writerow([data['KOL_username'], ', '.join(map(str, data['followers_usernames']))])

    print(f"Saved followers for {len(kol_followers_data)} KOLs to {output_file_path}.")



In [7]:
import csv
import time
from tweety.types import twDataTypes

async def crawl_followers_usernames(input_file_path, output_file_path, app, pages=1, wait_time=2):
    # Đọc danh sách screen_name từ file đầu vào
    with open(input_file_path, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        screen_names = [row['twitter_id'] for row in reader]
        screen_names = screen_names[500:1000] #thay số chỗ nà
    # Mở file lỗi để ghi log những user gặp lỗi
    with open("crawl_user_followers_error.txt", mode="a", encoding="utf-8") as error_file, \
         open(output_file_path, mode='a', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        
        # Ghi header vào file đầu ra (nếu file rỗng)
        if outfile.tell() == 0:  # Chỉ ghi header nếu file chưa có nội dung
            writer.writerow(['KOL_username', 'followers_usernames'])

        # Lặp qua từng KOL username để fetch followers
        for screen_name in screen_names:
            print(f"Fetching followers for user: {screen_name}")
            retry_count = 0

            while retry_count <= 3:  # Thử lại tối đa 3 lần
                try:
                    # Fetch followers bằng app Tweety
                    all_followers = await app.get_user_followers(
                        username=screen_name,
                        pages=pages,
                        wait_time=wait_time
                    )
                    
                    # Lấy danh sách username của followers
                    followers_usernames = [follower.username for follower in all_followers]
                    
                    # Lưu dữ liệu của user vừa crawl được vào file
                    writer.writerow([screen_name, ', '.join(map(str, followers_usernames))])
                    print(f"Saved followers for user: {screen_name}")
                    break  # Thoát khỏi vòng lặp retry nếu thành công
                except Exception as e:
                    retry_count += 1
                    print(f"Error fetching followers for user {screen_name}: {e}. Retrying ({retry_count}/3)...")
                    if retry_count > 3:
                        error_file.write(f"{screen_name}\n")
                    time.sleep(300)  # Chờ trước khi thử lại


In [8]:
input_file_path = "twitter_ids.csv"
output_file_path ="project_test_followers.csv"
await crawl_followers_usernames(input_file_path, output_file_path, app, pages=10, wait_time=10)

Fetching followers for user: KalmyAPP
Saved followers for user: KalmyAPP
Fetching followers for user: raft_fi
Saved followers for user: raft_fi
Fetching followers for user: gnosisPM
Saved followers for user: gnosisPM
Fetching followers for user: TheTNetwork
Saved followers for user: TheTNetwork
Fetching followers for user: parsiq_net
Saved followers for user: parsiq_net
Fetching followers for user: AcrossProtocol
Error fetching followers for user AcrossProtocol: [88] Rate limit exceeded
. Retrying (1/3)...
Error fetching followers for user AcrossProtocol: [88] Rate limit exceeded
. Retrying (2/3)...
Saved followers for user: AcrossProtocol
Fetching followers for user: HorizonProtocol
Saved followers for user: HorizonProtocol
Fetching followers for user: DomaniProtocol
Saved followers for user: DomaniProtocol
Fetching followers for user: CUDOS_
Saved followers for user: CUDOS_
Fetching followers for user: BuildOnLumia
Saved followers for user: BuildOnLumia
Fetching followers for user: a